In [21]:
import numpy as np
import sys
from tqdm import tqdm

nDimensions = 3
charge = 1
stepLength = 1.5
nParticles = 2
# step length and square inverse
h = 0.001
h2 = 1000000

nCycles = 1000000
thermalization = 1000

def local_energy(r, par):
    r_plus = np.zeros((nParticles, nDimensions))
    r_minus = np.zeros((nParticles, nDimensions))
    r_plus[:] = r
    r_minus[:] = r
    wave_function_minus = 0
    wave_function_plus = 0
    wave_function_current = wave_function(r, par)
    
    # Kinetic energy, brute force derivations
    kinetic_energy = 0
    for i in range(nParticles):
        for j in range(nDimensions):
            r_plus[i,j] += h
            r_minus[i,j] -= h
            wave_function_minus = wave_function(r_minus, par)
            wave_function_plus = wave_function(r_plus, par)
            kinetic_energy -= (wave_function_minus + wave_function_plus - 2 * wave_function_current)
            r_plus[i,j] = r[i,j]
            r_minus[i,j] = r[i,j]
    kinetic_energy = 0.5 * h2 * kinetic_energy / wave_function_current
    
    # Potential energy
    potential_energy = 0
    r_single_particle = 0
    for i in range(nParticles):
        r_single_particle = 0
        for j in range(nDimensions):
            r_single_particle += r[i,j] * r[i,j]
        potential_energy -= charge / np.sqrt(r_single_particle)
    
    # Contribution from electron-electron potential
    r12 = 0
    for i in range(nParticles):
        for j in range(i + 1, nParticles):
            r12 = 0
            for k in range(nDimensions):
                r12 += (r[i,k] - r[j,k]) * (r[i,k] - r[j,k])
            potential_energy += 1 / np.sqrt(r12)
    
    return kinetic_energy + potential_energy

# def wave_function(r, alpha):
#     argument = 0
#     for i in range(nParticles):
#         r_single_particle = 0
#         for j in range(nDimensions):
#             r_single_particle += r[i,j] * r[i,j]
#         argument += np.sqrt(r_single_particle)
#     return np.exp(-argument * alpha)

def wave_function(r, par):
    a, b = par
    r1 = 0.
    r2 = 0.
    r12 = 0.
    for k in range(nDimensions):
        r1  += r[0,k]**2
        r2  += r[1,k]**2
        r12 += (r[0,k] - r[1,k])**2
    r1  = np.sqrt(r1)
    r2  = np.sqrt(r2)
    r12 = np.sqrt(r12)
    return (np.exp(-a*r1-b*r2) + np.exp(-b*r1-a*r2))

def solve_VMC(par):
    rOld = np.zeros((nParticles, nDimensions))
    rNew = np.zeros((nParticles, nDimensions))
    waveFunctionOld = 0
    waveFunctionNew = 0
    energySum = 0
    energySquaredSum = 0
    deltaE = 0
    prob = 1.
    # initial trial positions
    rOld = stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)

    rNew = rOld
    # loop over Monte Carlo cycles
    accept = 0
    for cycle in tqdm(range(nCycles), file=sys.stdout, desc='VMC Cycles'):
        waveFunctionOld = wave_function(rOld, par)
        direction = (np.random.rand(nParticles, nDimensions)-0.5)
        for i in range(nParticles):
            for j in range(nDimensions):
                rNew[i,j] = rOld[i,j] + stepLength * direction[i,j]
            # Recalculate the value of the wave function
            waveFunctionNew = wave_function(rNew, par)
            # Check for step acceptance (if yes, update position, if no, reset position)
            if np.random.rand() <= np.float_power(waveFunctionNew/waveFunctionOld, 2):
                for j in range(nDimensions):
                    rOld[i,j] = rNew[i,j]
                waveFunctionOld = waveFunctionNew
                accept += 1
            else:
                for j in range(nDimensions):
                    rNew[i,j] = rOld[i,j]
            
            ### AVOID division error???????????????????????????????????????????????????????
            if wave_function(rOld, par) <= 1E-100:
                tqdm.write("triggered")
                rOld = stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)
                rNew = rOld + stepLength * (np.random.rand(nParticles, nDimensions) - 0.5)

            # update energies
            if cycle > thermalization:
                deltaE = local_energy(rNew, par)
                energySum += deltaE
                energySquaredSum += deltaE*deltaE
        if cycle % 20000 == 0: tqdm.write("energy:"+str(round(energySum/((cycle+1) * nParticles)*2*13.6,2))+" (eV) ; acceptance rate = "+str(round(accept/(cycle+1)/nParticles * 100,2))+"; alpha: "+str(alpha))
    energy = energySum/(nCycles * nParticles)*2*13.6
    energySquared = energySquaredSum/(nCycles * nParticles)
    # print(f"Energy: {energy*2*13.6}(eV), Variance: {energySquared-energy**2}")
    return energy


In [22]:
from multiprocessing import Pool

pool = Pool()

alphas = [(1.03922, 0.28322)]

energies = list(map(solve_VMC, alphas))

energy:0.0 (eV) ; acceptance rate = 100.0; alpha: 1    
energy:-14.86 (eV) ; acceptance rate = 80.2; alpha: 1                
triggered                                                            
energy:-15.31 (eV) ; acceptance rate = 79.25; alpha: 1               
energy:-14.88 (eV) ; acceptance rate = 78.81; alpha: 1               
energy:-14.65 (eV) ; acceptance rate = 78.45; alpha: 1               
energy:-14.49 (eV) ; acceptance rate = 78.33; alpha: 1               
energy:-14.39 (eV) ; acceptance rate = 78.17; alpha: 1                
energy:-14.32 (eV) ; acceptance rate = 78.08; alpha: 1                
energy:-14.28 (eV) ; acceptance rate = 78.03; alpha: 1                
energy:-14.23 (eV) ; acceptance rate = 77.95; alpha: 1                
energy:-14.18 (eV) ; acceptance rate = 77.86; alpha: 1                
VMC Cycles:  22%|██▏       | 218792/1000000 [00:31<01:53, 6879.18it/s]


KeyboardInterrupt: 

In [ ]:
energies

[-13.574631930610744]

In [ ]:
[(-2*alpha**2+4*charge*alpha-5/4*alpha)*13.6 for alpha in alphas]


TypeError: unsupported operand type(s) for ** or pow(): 'tuple' and 'int'

ModuleNotFoundError: No module named 'tqdm'

NameError: name 'rOld' is not defined

In [ ]:
import numba
